# 2025 데이터 크리에이터 캠프

@PHASE: Mission 1

@TEAM: 최후의 인공지능

## 1. Check GPU Availability

- GPU 번호 지정

In [ ]:
!nvidia-smi

In [ ]:
# Set CUDA Device Number
DEVICE_NUM = 0

from os import environ
environ["CUDA_VISIBLE_DEVICES"] = str(DEVICE_NUM)

## 2. 데이터셋 생성

- Ultralytics가 요구하는 dataset yaml 형식으로 데이터를 변환

### 2.1. json 파일을 Yolo label로 변환

In [ ]:
import json
from pathlib import Path
from PIL import Image


def convert_one_via_json_to_yolo_txt(
    via_json_path: Path,
    images_dir: Path = None,
    out_labels_dir: Path = Path("./dataset/train/yolo_labels"),
) -> int:
    """
    단일 VIA json -> YOLO txt 변환
    반환값: 생성된 박스 수(=라벨 라인 수). 0이면 파일 생성 안 함.
    """
    out_labels_dir.mkdir(parents=True, exist_ok=True)

    with via_json_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    out_path = out_labels_dir / (via_json_path.stem + ".txt")
    yolo_lines = []

    if isinstance(data, dict):
        items = data.values()
    elif isinstance(data, list):
        items = data
    else:
        print(f"[건너뜀] {via_json_path.name}: 알 수 없는 JSON 구조")
        return 0

    for item in items:
        if not isinstance(item, dict):
            continue
        filename = item.get("filename")
        if not filename:
            continue

        fa = item.get("file_attributes", {}) or {}
        img_w, img_h = fa.get("img_width"), fa.get("img_height")

        try:
            img_w, img_h = int(img_w), int(img_h)
        except (TypeError, ValueError):
            if images_dir:
                img_path = images_dir / filename
                if img_path.exists():
                    with Image.open(img_path) as im:
                        img_w, img_h = im.size
            if not img_w or not img_h:
                print(f"[건너뜀] {via_json_path.name} / {filename}: 이미지 크기 불명")
                continue

        for r in item.get("regions", []):
            sa = (r or {}).get("shape_attributes", {}) or {}
            if sa.get("name") != "rect":
                continue

            x = float(sa.get("x", 0))
            y = float(sa.get("y", 0))
            w = float(sa.get("width", 0))
            h = float(sa.get("height", 0))
            if w <= 0 or h <= 0:
                continue

            cx = (x + w / 2) / img_w
            cy = (y + h / 2) / img_h
            nw = w / img_w
            nh = h / img_h

            yolo_lines.append(f"0 {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")

    if yolo_lines:
        out_path.write_text("\n".join(yolo_lines), encoding="utf-8")
        print(f"[OK] {out_path}  ({len(yolo_lines)} boxes)")
        return len(yolo_lines)
    else:
        print(f"[정보] {via_json_path.name}: 박스 없음")
        return 0


def convert_folder_via_jsons_to_yolo_txts(
    json_dir: str,
    images_dir: str = None,
    out_labels_dir: str = "./dataset/train/yolo_labels",
    pattern: str = "*.json"
):
    json_dir = Path(json_dir)
    images_dir = Path(images_dir) if images_dir else None
    out_labels_dir = Path(out_labels_dir)
    out_labels_dir.mkdir(parents=True, exist_ok=True)

    json_files = sorted(json_dir.glob(pattern))
    if not json_files:
        print(f"[경고] JSON 파일이 없습니다: {json_dir}/{pattern}")
        return

    total_files = 0
    total_boxes = 0
    for jf in json_files:
        try:
            n = convert_one_via_json_to_yolo_txt(
                via_json_path=jf,
                images_dir=images_dir,
                out_labels_dir=out_labels_dir
            )
            total_files += 1
            total_boxes += n
        except Exception as e:
            print(f"[에러] {jf.name}: {e}")

    print(f"\n=== 변환 완료 ===")
    print(f"- 처리한 JSON: {total_files}개")
    print(f"- 생성된 박스(라인): {total_boxes}개")
    print(f"- 출력 폴더: {out_labels_dir.resolve()}")


convert_folder_via_jsons_to_yolo_txts(
    json_dir="./dataset/valid/labels",      # JSON들이 모여있는 폴더
    images_dir="./dataset/valid/images",    # (선택) 이미지 폴더
    out_labels_dir="./dataset/valid/yolo_labels"  # TXT를 저장할 다른 폴더
)

[OK] dataset/valid/yolo_labels/K3A_CHN_20161112052404_15.txt  (1 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170115051130_1.txt  (1 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170123052151_1.txt  (1 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170123052151_14.txt  (1 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170123052151_22.txt  (2 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170123052151_8.txt  (4 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170210052214_14.txt  (1 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170210052214_15.txt  (1 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170423052236_20.txt  (1 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170423052236_3.txt  (4 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170723050144_20.txt  (3 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170723050144_25.txt  (3 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170723050144_28.txt  (5 boxes)
[OK] dataset/valid/yolo_labels/K3A_CHN_20170723050144_3.txt  (2 boxe

### 2.2. yaml 파일 생성

In [6]:
import yaml

# 데이터셋 정보 설정
dataset = {
    'path': '/root/jupyter/K-ICT_data_creator/dataset/',
    'train': 'train/images',  
    'val': 'test/images', 
    'test': 'test/images',
    'nc': 1,  
    'names': ['chimney']
}

# YAML 파일로 저장
with open('dataset.yaml', 'w') as file:
    yaml.dump(dataset, file, sort_keys=False)  

## 3. 모델 학습 및 평가

### 3.1 Yolov8 Model Training

In [ ]:
from ultralytics import YOLO

# yolov8s 모델 불러오기 (자동 다운로드됨)
model = YOLO("yolov8s.pt")

results = model.train(
    data='dataset.yaml', 
    seed=0, # 데이터 시드 고정 
    epochs=300, 
    imgsz=512, # 원본 이미지 해상도 사용
    project='results',
    name='yolov8s_512',
    tracker="wandb",
    val=True
)

New https://pypi.org/project/ultralytics/8.3.209 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.201 🚀 Python-3.8.10 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100 80GB PCIe MIG 7g.80gb, 81051MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=MIG-64c256ce-5475-5deb-8bca-2cb0fca126f3, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=Fal

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jhm9808 (diplab_sar). Use `wandb login --relogin` to force relogin


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

lr/pg0,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▄▇▇████████████████████████████████████
metrics/mAP50-95(B),▁▃▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████
metrics/precision(B),▁▄▆▆▇▆▇▇▇▇▇█▇▇▇▇█▇▇▇▇▇▇▇▇▇██████████████
metrics/recall(B),▁▅▆▇▇▇▇▇████████████████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▆▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁


### 3.1 Yolov8 Model Evaluation

In [ ]:
from ultralytics import YOLO

model = YOLO("./results/yolov8s_512/weights/last.pt")  # 마지막으로 저장된 모델 가중치 사용

In [ ]:
test_metrics = model.val(
    source="dataset.yaml",
    split='test',
    # 지정된 inference setting
    conf=0.25,   # confidence threshold
    iou=0.50,    # NMS IoU threshold
    max_det=100  # max detections
)

print("Test metrics:", test_metrics)

Ultralytics 8.3.184 🚀 Python-3.8.10 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.2±0.1 ms, read: 251.2±62.1 MB/s, size: 90.9 KB)


val: Scanning /root/jupyter/K-ICT_data_creator/dataset/valid/labels.cache... 1006 images, 0 backgrounds, 0 corrupt: 100% 1006/1006 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 63/63 [00:04<00:00, 14.20it/s]


                   all       1006       1322      0.989       0.99      0.993      0.891
Speed: 0.2ms preprocess, 1.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val8
Test metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f8f064cc370>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.0320